In [1]:
# Day 1
# Part one
import numpy as np
data = np.loadtxt('day1.txt', dtype=int)
for j in range(2):
    data[:,j] = sorted(data[:,j])
print(np.sum(np.abs(data[:,1] - data[:,0])))

# Part two
from collections import defaultdict
numbers, counts = np.unique(data[:,1], return_counts=True)
look_up_dict = defaultdict(int)
look_up_dict.update({number:count for number, count in zip(numbers, counts)})
print(sum(d*look_up_dict[d] for d in data[:,0]))

1834060
21607792
